# Simulation numérique – ProgFest 2025
---
Par Arthur Légaré, le 5 mars 2025.

*Le masculin est utilisé pour alléger le texte.*

# Pourquoi simuler en physique?
---

Curieux de nature, le physicien est porté à expérimenter avec toutes sortes de cossins à sa disposition. Toutefois, avec la maîtrise du langage mathématique, sa curiosité vient généralement à bout des expériences immédiatement accessibles : il devient donc urgent d'agrandir le carré de sable!

Venons-y : la physique comporte son lot de phénomènes qui sont décrits par des équations complexes, souvent difficiles, voire impossibles, à résoudre analytiquement. C'est là que la simulation numérique entre en jeu, notamment pour :
1) explorer des systèmes trop complexes pour admettre une solution exacte;
2) visualiser des dynamiques évolutives et leurs comportements émergents;
3) tester des hypothèses et comparer avec des résultats expérimentaux.

Certains puristes pourraient y voir là les contours d'une sacrification de la physmath sur l'autel de la *force brute*. Mais attention : l'usage des simulations ne relègue pas aux oubliettes la physmath. Au contraire, nous verrons plutôt que cette dernière demeure l'outil privilégié pour affuter notre intuition, guidant ainsi nos explorations numériques et facilitant l'interprétation des résultats.


# 1. Plan de match
---

Nous verrons deux exemples de simulations numériques portant sur des sujets variés. Au terme de cet atelier, nous devrions être en mesure de mettre sur pied une recette pour simuler avec succès (voir annexe Z).

**a) Exercices en rafale** – Passer de l'équation à un modèle simulable.

**b) Échanges thermiques dans un bâtiment** – Comparaison entre différentes stratégies de chauffage.



In [ ]:
# Importons quelques librairies utiles.
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
#import inc.functions as fc
import scipy as sp
from numba import njit
import matplotlib.animation as animation
from IPython.display import display, HTML

plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['font.size'] = 16

%matplotlib inline  # Pour éviter que les animations ne s'ouvrent dans une autre fenêtre.

# 2. Exercices
---

La prochaine section présente deux exercices pour vous permettre de pratiquer le passage de l'équation au code!

## a) Oscillateur harmonique amorti

Servi à toutes les sauces – que ce soit en mécaniques classique, lagrangienne et quantique ou encore en physique statistique –, l'oscillateur harmonique est un système dynamique omniprésent en physique. Son équation différentielle, avec un amortissement $b$, devrait vous être des plus familières :
$$
\boxed{m\frac{\text{d}^2 x(t)}{\text{d}t^2} + b\frac{\text{d}x(t)}{\text{d}t} + kx(t) = 0},
$$
où on reconnait $x$ comme une position quelconque, $k$ comme la fréquence propre de l'oscillateur et $b$ comme le coefficient d'amortissement.

### i) Décidons des paramètres de l'oscillateur harmonique amorti

In [ ]:
m = 1.0  # Masse
k = 1.0  # Constante de raideur du ressort
b = 0.1  # Amortissement (mettre 0 pour un ressort idéal)

### ii) Choisissons les conditions initiales du problème

In [ ]:
# Conditions initiales du problème.
x0 = 1.0  # Position initiale
v0 = 0.0  # Vitesse initiale

### iii) Mathématisons le problème

a) On commence par obtenir l'expression de l'accélération instantanée, ce qui permettra de mettre à jour la vitesse : $\Delta v \approx a \Delta t$.
$$m\ddot{x}(t) + b\dot{x}(t) + kx(t) = 0 \Longleftrightarrow \ddot{x}(t) = \frac{1}{m}\Big (- b\dot{x}(t) - kx(t) \Big)$$
L'accélération instantanée de cet oscillateur va donc comme : 
$$\boxed{a(t) =  -\frac{1}{m}\Big (b\dot{x}(t) + kx(t) \Big)}.$$



b) Ensuite, la position pourra être mise à jour successivement par cinématique avec la relation $\Delta x \approx v\Delta t$.


### iv) Passons de l'équation au code

 Voici l'idée générale, transcrite en pseudo-code, d'une intégration reposant sur la **méthode d'Euler** appliquée à cette dynamique :
```
FONCTION update_state(x, v, dt, m, k, b)
    // Estimation de l'accélération selon l'équation du mouvement.
    a ← -(b*v + k*x)/m
    
    // Mettre à jour la vitesse avec la méthode d'Euler.
    v_new ← v + a*dt
    
    // Mettre à jour la position avec la nouvelle vitesse (par cinématique).
    x_new ← x + v_new*dt
    
    // Retourner la nouvelle position et la nouvelle vitesse.
    RETOURNER (x_new, v_new)
FIN FONCTION
```


In [ ]:
# Pour commencer, on intègre simplement avec la méthode d'Euler.
def integrer_OHA_euler(T, m, k, b, x0, v0):
    x_integree = np.zeros(T)
    t_integree = np.zeros(T)
    v_integree = np.zeros(T)
    x = x0
    v = v0
    temps = 0 # On harcode un temps nul au début.

    for t in range(T):
        a = (-b*v - k*x) / m  # Accélération instantanée due à la force de rappel et à l'amortissement
        v += a * dt  # Nouvelle vitesse
        x += v * dt  # Nouvelle position
        temps += dt # Incrément temporel
        x_integree[t] = x
        t_integree[t] = temps
        v_integree[t] = v

    return t_integree, x_integree, v_integree


### v) Intégrons!

In [ ]:
# Paramètres d'intégration.
dt = 0.05  # Pas de temps
T = 400  # Nombre de frames pour l'animation

print(f"Nous allons intégrer sur {np.round(T*dt, 3)} s")

t_solution, x_solution, v_solution = integrer_OHA_euler(T, m, k, b, x0, v0)

### vi) Visualisons les résultats


In [ ]:
_, ax = plt.subplots(1,1, figsize=(10,5))
ax.plot(t_solution, x_solution, color="darkBlue", linewidth=2)
plt.xlabel(r"$t$ [s]")
plt.ylabel(r"$x(t)$ [m]")
ax.spines[['right', 'top']].set_visible(False)
plt.show()

In [ ]:
_, ax = plt.subplots(1,1, figsize=(10,5))
ax.plot(x_solution, v_solution, color="darkBlue", linewidth=2)
plt.xlabel(r"$x(t)$ [m]")
plt.ylabel(r"$\dot{x}(t)$ [m/s$^2$]")
ax.spines[['right', 'top']].set_visible(False)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_xlim(-x0*1.5, x0*1.5) # Domaine en x visible.
ax.set_ylim(-0.2, 0.2) # Domaine en y visible.
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Oscillateur harmonique amorti")

# Ligne représentant le ressort
spring, = ax.plot([], [], color="black", linewidth=3)
mass, = ax.plot([], [], color="darkBlue", marker=".", markersize=100)  # Masse attachée

def init():
    spring.set_data([], [])
    mass.set_data([], [])
    return spring, mass

def update(frame):
    x = x_solution[frame]
    spring.set_data(np.linspace(-x0*1.5, x, 10), np.zeros(10))  # Déforme le ressort
    mass.set_data([x], [0])  # Déplace la masse
    return spring, mass

# Affichage de l'animation.
ani = animation.FuncAnimation(fig, update, frames=np.arange(0,T,2), init_func=init, interval=50, blit=True)
display(HTML(ani.to_jshtml()))

## b) Modèle de Kuramoto

Soit le modèle de Kuramoto, un modèle de $N$ oscillateurs couplés, souvent utilisé pour étudier les phénomènes de synchronisation collective dans des systèmes complexes. Le modèle s'exprime par l'équation différentielle suivante :
$$
\boxed{\dot{\theta}_k = \omega_k - \frac{g}{N} \sum_{j=1}^N \sin \left(\theta_k - \theta_j \right) \hspace{12pt} \forall\hspace{6pt} k\in\{1,...,N\}},
$$
où :
* $g$ est le couplage global du système (on peut même faire appel à une matrice $G=[g_{ij}]$ pour un couplage non uniforme),
* $\omega$ est la fréquence naturelle des oscillateurs, 
* $\theta_k$, la phase de l'oscilleur $k$.

Le modèle de Kuramoto est particulièrement pertinent pour comprendre les mécanismes de synchronisation dans des systèmes complexes, où une grande d'entités interagissent entre elles et se synchronisent spontanément en dépit de variabilité individuelle, comme c'est observé dans des phénomènes biologiques, sociaux et physiques. [3].

In [ ]:

# i) Paramètres du modèle.

N = 20  # Nombre d'oscillateurs
g = 0.5  # Couplage global
dt = 0.01  # Pas de temps
t_f = 15 # Temps final de simulation

# ii) Initialisation des phases (theta) et des fréquences naturelles (omega).

theta_0 = np.random.uniform(0, 2*np.pi, N)  # Phases initiales aléatoires.
omega = np.random.normal(2, 1, N)  # Fréquences naturelles de moyenne 2 et d'écart-type 1.

# iii) et iv) Simulation numérique (intégration avec solve_ivp pour de meilleurs résultats!).

t_vec = np.linspace(0, t_f, int(t_f/dt), endpoint=True) # Vecteur des temps auxquels évaluer l'intégrale

@njit
def kuramoto_deriv(t, theta, g, omega):
    N = omega.shape[0]
    theta_dot = np.zeros(N)
    for k in range(N):
        theta_dot_k = omega[k] - g/N * sum([np.sin(theta[k] - theta[j]) for j in range(N)])
        theta_dot[k] = theta_dot_k
    return theta_dot

solution = sp.integrate.solve_ivp(kuramoto_deriv, t_span=(0, t_f), y0=theta_0, args=(g, omega), t_eval=t_vec, rtol=1e-6, method="RK45")

theta_solution = solution.y

In [ ]:

# v) Visualisation des résultats (une animation va nous satisfaire pour ce modèle...).

import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 30  # en MB

sous_echant_frames_anim = 5 # On retient une frame sur [] pour l'animation.

fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xlim(-1.2, 1.2)
ax.set_ylim(-1.2, 1.2)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title(f"Modèle de Kuramoto : {N} oscillateurs couplés")
ax.spines[["left", "right", "top", "bottom"]].set_visible(False)

circle = plt.Circle((0, 0), 1, color="black", fill=False, linewidth=1.2)  # Cercle de rayon unitaire.
ax.add_patch(circle)

points, = ax.plot([], [], marker=".", linestyle="None", color="darkGreen", markersize=12)  # Points des oscillateurs.
com, = ax.plot([], [], marker="+", color="red", markersize=15)  # Point pour le centre de masse.
trajectory, = ax.plot([], [], color="blue", linewidth=1)  # Trajectoire du centre de masse.

x_com_trace = []
y_com_trace = []

def init():
    points.set_data([], [])
    com.set_data([], [])
    trajectory.set_data([], [])
    return points, com, trajectory

def update(frame):
    x = np.cos(theta_solution[:, frame])
    y = np.sin(theta_solution[:, frame])
    points.set_data([x], [y])

    x_center = np.mean(x)
    y_center = np.mean(y)
    x_com_trace.append(x_center)
    y_com_trace.append(y_center)
    com.set_data([x_center], [y_center])  # Calcul du centre de masse
    trajectory.set_data([x_com_trace], [y_com_trace])  # Mise à jour de la trajectoire

    return points, com, trajectory

ani = animation.FuncAnimation(fig, update, frames=range(0, len(t_vec), sous_echant_frames_anim), init_func=init, interval=60, blit=True)
display(HTML(ani.to_jshtml()))

# 3. **Application à un cas réel** : Échanges <span style="color:blue">t</span><span style="color:deepskyblue">h</span><span style="color:mediumslateblue">e</span><span style="color:mediumpurple">r</span><span style="color:mediumorchid">m</span><span style="color:orchid">i</span><span style="color:hotpink">q</span><span style="color:deeppink">u</span><span style="color:crimson">e</span><span style="color:red">s</span> dans un bâtiment
---

Quel est le scénario de chauffage est le **moins énergivore** : une désactivation du radiateur pour la journée ou une légère diminution de la température cible? Forts de notre intuition, nous savons qu'il faut minimiser l'énergie associée au chauffage.
$$
\text{À minimiser : } E_{\mathrm{chauffage}} = \int_{t_i}^{t_f} P_{\mathrm{radiateur}}(\tau)\text{d}\tau.
$$

<img src="inc/radiateur_off.png" width=1000 height=450 />


### Hypothèses
1) **Loi de Fourier** pour la conduction de chaleur $\frac{\text{d}U(t)}{\text{d}t} = \frac{\kappa A}{d}\Delta T$, est le mode dominant de transfert thermique [10].

2) **Gaz parfait** : l'énergie interne,
$
\text{d}U = nC_v \text{d}T,
$
ne dépend que de la température de l'air.

3) **Temps de relaxation** thermique : le temps que prend une pièce pour uniformiser sa température est négligeable comparativement au temps caractéristique des transferts thermiques entre pièces, si bien que
$
\tau_{\mathrm{relax}} \ll \tau_{\mathrm{transfert}},
$
ce qui permet de désigner la température d'une pièce $i$ simplement par $T_i(t)$.

4) On considère qu'aucun échange d'air n'a lieu entre les pièces et que la quantité initiale est donc confinée à chaque pièce.



### Mathématisation du problème


En combinant l'expression de variation d'énergie interne d'un gaz parfait,
$$
\text{d}U = nC_v \text{d}T,
$$
et la loi de Fourier pour la conduction thermique,
$$
\frac{\text{d}U}{\text{d}t} = \frac{\kappa A}{d}\Delta T
$$
$$
\Longleftrightarrow \text{d}U(t)  = \frac{\kappa A}{d}\Delta T\text{d}t,
$$
nous obtenons
$$
\text{d}U = \text{d}U \implies nC_v \text{d}T(t) = \frac{\kappa A}{d}\Delta T(t) \text{d}t 
$$
$$
\Longleftrightarrow \boxed{nC_v \frac{\text{d}T(t)}{\text{d}t} = \frac{\kappa A}{d}\Delta T(t)}.
$$
Au passage, il est intéressant de remarquer que cette relation est **équivalente à la loi de refroidissement des corps**, formulée par Newton en 1701 [4].

En ce qui nous concerne, comme le taux de variation de température d'une pièce $i$ causé par le transfert thermique avec la pièce $j$ est proportionnel à la différence $\Delta T_{ij}(t) \equiv -(T_i(t)-T_j(t))$, on obtient
$$
n_iC_v \frac{\text{d}T_i(t)}{\text{d}t} = -\sum_{j\neq i} \underbrace{\frac{\kappa_{ij} A_{ij}}{d_{ij}}}_{\equiv k_{ij}}\Big(T_i(t) - T_j(t)\Big),
$$
où il faut sommer sur les $j\in\{1,...,N\}\backslash \{i\}$ autres pièces, qui agissent à titre de "tributaires" thermiques.

Nous ne sommons pas sur $j=i$ étant donné qu'on n'assume aucune conductance de la pièce $i$ avec elle-même : $k_{ii} = 0 \; \forall i \in \mathbb{N}$. La diagonale de la matrice d'adjacence est donc nulle, ce qui nécessite le facteur $(1-\delta_{ij})$, de telle sorte que
$$
\boxed{n_iC_v \frac{\text{d}T_i(t)}{\text{d}t}  = -\sum_{j=1}^N k_{ij} \Big(T_i(t) - T_j(t)\Big) (1-\delta_{ij}) }.
$$
Cette situation correspond bien à celle d'un bâtiment de $N$ pièces, lesquelles présentent des conductances thermiques encapsulées par la matrice d'adjacence
$$
K = \left [ k_{ij}\right ].
$$


#### Exemple de matrice d'adjacence $A=\left[a_{ij}\right]$ d'un graphe non orienté (symétrique) à $N=10$ noeuds
<p float="left">
    <img src=https://mathinsight.org/media/image/image/small_undirected_network.png width = "300"/>
    <img src="inc/right_arrow.png" width = "300" height = "300"/>
    <img src=https://mathinsight.org/media/image/image/small_undirected_network_numbered.png width = "300"/>
</p>

Dans notre cas, toutefois, nous voulons éviter d'avoir à exprimer l'ÉDO en termes de différences en température $\Delta T_{ij}$, puisque cela empêche l'<span style="color:cyan">**écriture sous forme matricielle!**</span>

#### Écriture sous forme matricielle de l'ÉDO

Pour pouvoir réécrire l'équation seulement en termes de vecteurs et de matrices, on cherche donc une forme condensée exploitant le produit matriciel :
$$
\underbrace{n_iC_v}_{\equiv C_{ii}} \frac{\text{d}T_i(t)}{\text{d}t}  = -\sum_{j=1}^N k_{ij} \Big(T_i(t) - T_j(t)\Big) (1-\delta_{ij}) \Longrightarrow C\frac{\text{d}\mathbf{T}(t)}{\text{d}t} = - L\mathbf{T}(t),
$$
où la matrice d'«inertie» thermique $C$ ainsi que la matrice de couplage globale $L=[l_{ij}]$, qui encapsule tous les transferts thermiques par conduction, sont introduites. La question qui se pose maintenant est : est-ce possible et de construire une telle matrice ($L$) et, le cas échéant, de quoi sera-t-elle constituée?


Pour commencer, intéressons-nous au côté droit de l'ÉDO et distinguons les deux termes contenus dans le terme général de la somme.
$$
\sum_{j=1}^N k_{ij}\Big(T_i(t) - T_j(t))(1-\delta_{ij}) = \sum_{j=1}^N k_{ij}T_i(t)(1-\delta_{ij}) - \sum_{j=1}^N k_{ij}T_j(t)(1-\delta_{ij})
$$

Pour retrouver la forme d'un produit matriciel avec un éventuel vecteur de températures $\mathbf{T}$, on doit mettre en évidence le facteur $T_i(t)$ au premier terme, pour les éléments diagonaux de $L$. De même, le facteur $T_j(t) (1-\delta_{ij})$ doit être mis en évidence du deuxième terme pour les éléments hors diagonale de la matrice $L$, ce qui donne
$$
\underbrace{\Bigg [\sum_{j=1}^N k_{ij}(1-\delta_{ij}) \Bigg]}_{\equiv l_{ii}}T_i(t) + \sum_{j=1}^N \underbrace{\Bigg[-k_{ij}\Bigg]}_{\equiv l_{ij}}T_j(t)(1-\delta_{ij}).
$$

Il est maintenant possible de faire correspondre tout le reste des termes à l'expression de $L_{ij}$, selon que $i=j$ (diagonale) ou que $i\neq j$ (hors diagonale), si bien que
$$
l_{ij} = \underbrace{\Bigg[ \sum_{\xi=1}^N k_{i\xi} (1-\delta_{i\xi}) \Bigg]}_{\text{Diagonale}}\delta_{ij} + \underbrace{\Bigg [-k_{ij} \Bigg]}_{\text{Hors diagonale}}(1-\delta_{ij}) \hspace{12pt} \Longleftrightarrow \hspace{12pt} l_{ij} = \begin{cases} \sum_{\xi=1}^N k_{i\xi} (1-\delta_{i\xi}) \hspace{12pt} \text{ si } j=1, \\ -k_{ij} \hspace{12pt} \text{ si }j\neq i.\end{cases}
$$


Forts de cette expression pour $L$, on peut maintenant sans tracas exprimer l'ÉDO sous forme purement martricielle comme 
$$
\boxed{C\frac{\text{d}\mathbf{T}(t)}{\text{d}t} = - L\mathbf{T}(t)}.
$$

**Fait intéressant :** cette matrice $L$ qui combine des conductances thermiques correspond à une **matrice de Laplace** [5] où les éléments sur la diagonale sont les degrés pondérés entrants. Une telle matrice a la particularité d'être définie positive (c'est-à-dire qu'elle ne possède que des valeurs propres positives) [6]. Ce faisant, sans l'intervention de radiateurs, les températures des pièces sont vouées à se stabiliser de manière monotone vers des valeurs d'équilibre thermique, ce qui se démontre aisément dans l'*espace propre* de $C^{-1}L$, quantité matricielle également définie positive (voir annexe C pour détails).


Maintenant que nous avons formulé mathématiquement les échanges thermiques entre pièces $i$ et $j$, nous pouvons introduire les deux autres composantes-clés au modèle : la puissance de chauffage des radiateurs $\mathbf{P}(t)$ pour les $N$ pièces ainsi que le transfert thermique avec l'extérieur, de température $T_{\mathrm{ext}}$, qui peut être vu comme le "chauffage du deillors" (en bon français).

#### Sous forme scalaire :
$$
\boxed{C_i \frac{\text{d}T_i(t)}{\text{d}t}=  \underbrace{-\sum_{j=i}^{N}L_{ij}T_j(t)}_{\text{Transferts thermiques entre les pièces}} +\underbrace{P_i(t)}_{\text{Chauffage}}  - \underbrace{K_{\mathrm{ext},i}(T_i(t) - T_{\mathrm{ext}})}_{\text{Pertes vers l'extérieur}}}
$$

#### Sous forme matricielle :
$$
\boxed{\frac{\text{d}\mathbf{T}(t)}{\text{d}t} = \underbrace{C^{-1}}_{\text{Inertie thermique}} \Big(- L\mathbf{T}(t) + \mathbf{P}(t)  - K_{\mathrm{ext}}\left(\mathbf{T}(t) - \mathbf{T}_{\mathrm{ext}} \right) \Big)}
$$

### Estimation de la matrice d'adjacence à partir de données réelles

<img src="inc/plan_appart.png" width=1000 height=450 />

**Légende :**
- <span style="color:pink">Mur isolant parfait (0 W/m*K)</span>
- <span style="color:cyan">Mur vers l'extérieur (0.05 W/m*K)</span>
- <span style="color:yellow">Mur de gypse (0.2 W/m*K)</span>


In [ ]:
"""
Choissons judicieusement les paramètres pour estimer la conductance 
à partir de la conductivité, de la superficie et de l'épaisseur des murs communs.
"""

# Paramètres réalistes pour estimer par après la conductance thermique.
kappa_ext = 0.03  # Conductivité thermique plausible pour mur extérieur [W/m*K]
kappa = 0.2       # Conductivité thermique plausible pour du gypse [W/m*K]
d_ext = 0.3       # Épaisseur des murs extérieurs [m]
d = 0.1           # Épaisseur des murs entre les pièces [m]
hauteur = 3       # Hauteur du plafond [m]


In [ ]:

"""
Définissons la matrice répertoriant les superficies (en m^2).
Celle-ci est symétrique puisque les aires sont partagées.
"""

A = np.zeros((6,6)) # Par défaut, les pièces ne sont pas connectées.
A[0,5] = 2.5*hauteur # Entre pièces 1 et 6
A[1,5] = 2.5*hauteur # Entre pièces 2 et 6
A[1,2] = 4.0*hauteur # Entre pièces 2 et 3
A[2,3] = 4.0*hauteur # Entre pièces 3 et 4
A[3,4] = 4.0*hauteur # Entre pièces 4 et 5
A += A.T # Symétrisons la matrice des aires.
print(A)

In [ ]:

"""
1) Définissons le vecteur contenant les superficies de façade extérieure.
2) De même, définissons le vecteur contenant les volumes des pièces.
"""
A_ext = hauteur*np.array([4+5, 4+5, 5, 4, 5+4, 2.5])
volumes_pieces = hauteur*np.array([4*5, 4*5, 4*5, 4*4, 5*4, 2.5*2.5])

In [ ]:
"""
Définissons la matrice des conductivités thermiques (en W/m*K).
Ce ne sont pas toutes les pièces qui sont connectées entre elles.

 - Rose = mur isolant parfait (0 W/m*K)
 - Bleu = mur vers l'extérieur (0.1 W/m*K)
 - Jaune = mur de gypse (0.2 W/m*K)
"""

conductivites = kappa*np.ones((6,6))

In [ ]:
"""
1) Calculons la matrice de conductance (dite d'«adjacence»).
2) De même pour le vecteur de conductance avec l'extérieur.
"""

K = np.multiply(A,conductivites)/d
K_ext = A_ext*kappa_ext/d_ext

print("Matrice d'adjacence K :")
print(K)
print("Vecteur de conductances vers l'extérieur :")
print(K_ext)

In [ ]:
"""
Calculons la matrice laplacienne (L) de conductance à partir de la matrice d'adjacence (K).
"""

L = np.zeros_like(K)
for i in range(L.shape[0]):
    for j in range(L.shape[1]):
        if i != j:
            L[i, j] = -K[i, j] # Hors diagonale
        else:
            L[i, j] = np.sum(K[i, :]) # Sur diagonale

print("Matrice laplacienne de conductance thermique L :")
print(L)


In [ ]:
rho_air = 1.2 # [Kg/m^3]
masse_molaire_air = 29e-3 # [Kg/mol]
densite_molaire = rho_air / masse_molaire_air
C = densite_molaire*np.diag(volumes_pieces)  # Matrice des "inerties thermiques" (chaleurs spécifiques par nombre de moles).
print(C)

#### Planification d'un horaire de chauffage


In [ ]:
# Temps d'intégration.
t_f = 20000 # [s]
t_domaine = np.linspace(0, t_f, t_f)
dt = t_domaine[1] - t_domaine[0]

T_0 = 20.0*np.ones(6) # Températures initiales (CI).
T_ext = -15.0 # Fait frette deillors!

def T_ext_fonction(t):
    return np.ones(6)*T_ext # Fonction constante, (journée pas de soleil, mettons)

T_off = 0 # Température jugée comme étant "off"
piece_off = 2 # Salon
off_temps_debut = int(0.05*t_f) # Temps à partir duquel on repart le chauffage en fin de journée
off_temps_fin = int(0.5*t_f) # Temps à partir duquel on met un radiateur à off


In [ ]:

T_horaire = np.array([T_0 for _ in range(t_domaine.shape[0])])
T_horaire_off = T_horaire.copy()

print(T_horaire_off.shape)


for i,temps_actuel in enumerate(t_domaine):
    if temps_actuel > off_temps_debut and temps_actuel < off_temps_fin:
        T_horaire_off[i,piece_off] = T_off


def T_cible_fonction(t_actuel, t_vect, T_hor):
    index = np.searchsorted(t_vect, t_actuel, side="right") - 1
    index = max(0, min(index, T_hor.shape[0] - 1))
    return T_hor[index, :]

plt.figure(figsize=(8,3))
plt.plot(t_domaine/3600, T_horaire[:,piece_off], label="Sans OFF", color="darkRed")
plt.plot(t_domaine/3600, T_horaire_off[:,piece_off], label="Avec OFF", color="darkGreen")
plt.xlabel("Temps [h]")
plt.ylabel(r"Temp. ciblée [$^{\circ}$C]")
plt.legend()
plt.show()


#### Usage d'un contrôleur PID
Voici les ancêtres des mécanismes de contrôle que nous allons implémenter dans cette simulation [2]. Il s'agit de régulateurs à boules [7], qui ont été inventés pour régulariser la vitesse de rotation des premières machines à vapeur. Aujourd'hui, le principe de base de ces mécanismes est utilisé dans toutes sortes de dispostifs de stabilisation, allant du *cruise-control* sur nos routes à nos thermostats, dans nos maisons. C'est cette dernière application qui nous intéresse.
<p float="left">
    <img src="inc/regulateur_machine_vapeur.png" width=600 height=450 />
    <img src="inc/regulateur_boules_zoom.png" width=350 height=450 />
</p>

Plus spécifiquement, les contrôleurs PID (Proportionnel-Intégral-Dérivé) sont des outils essentiels pour réguler des systèmes dynamiques. Ils permettent de maintenir une variable de sortie à une valeur désirée en ajustant continuellement les entrées du système. Le contrôleur PID combine trois actions de contrôle distinctes :

1. **Action proportionnelle (P)** : Cette action est proportionnelle à l'erreur actuelle, c'est-à-dire la différence entre la valeur mesurée et la valeur cible. Elle permet une réponse rapide aux variations de l'erreur.
2. **Action intégrale (I)** : Cette action est proportionnelle à l'intégrale de l'erreur sur le temps. Elle corrige les erreurs accumulées sur le long terme, éliminant ainsi les erreurs résiduelles.
3. **Action dérivée (D)** : Cette action est proportionnelle à la dérivée de l'erreur. Elle anticipe les variations futures de l'erreur, apportant une stabilité et réduisant les oscillations.

La combinaison de ces trois actions permet au contrôleur PID de fournir une réponse équilibrée et efficace, en minimisant l'erreur et en stabilisant le système. Les gains proportionnel ($K_p$), intégral ($K_i$) et dérivé ($K_d$) doivent être ajustés pour chaque application spécifique afin d'obtenir les performances souhaitées.

In [ ]:
%run simul_chauffage.py

# Définissons les gains du contrôleur PID
K_p = 10
K_i = 0.1
K_d = 1

pid = PIDController(T_cible_fonction(t_domaine[0], t_domaine, T_horaire), K_p, K_i, K_d)

def P_fonction(t, t_vect, T, T_hor):
    T_cible = T_cible_fonction(t, t_vect, T_hor)
    if np.any(T_cible != pid.T_target):
        pid.update_T_target(T_cible)
        pid.reset()  # Réinitialiser l'intégrale de l'erreur et l'erreur précédente
    return pid.calcul_puissance(T, dt)

T_sans_off, P_sans_off = simule_modele_thermique(C, L, K_ext, T_ext_fonction,
    lambda t, T: P_fonction(t, t_domaine, T, T_horaire), T_0, t_domaine)

T_avec_off, P_avec_off = simule_modele_thermique(C, L, K_ext, T_ext_fonction,
    lambda t, T: P_fonction(t, t_domaine, T, T_horaire_off), T_0, t_domaine)

In [ ]:
# Affichage des résultats.
legends = [ f"Pièce {i+1}" for i in range(L.shape[0])]

plt.figure(figsize=(10, 8))
plt.subplot(2, 1, 1)
for i in range(T_avec_off.shape[1]):
    plt.plot(t_domaine, T_avec_off[:,i], linewidth=2, label=legends[i])
plt.plot(t_domaine, T_horaire_off[:,piece_off], linestyle="--", linewidth=2,color="black", label="Horaire OFF")
plt.legend(loc="upper right")
plt.xlabel("Temps [s]")
plt.ylabel(r"Température [$^{\circ}$C]")
plt.title("Simulation des transferts thermiques")

plt.subplot(2, 1, 2)
plt.plot(t_domaine, P_avec_off, linestyle="--", linewidth=2)
plt.plot(t_domaine, P_sans_off, linestyle="-", linewidth=2)
extra_ylim = 30
plt.ylim(min([np.min(P_avec_off),np.min(P_sans_off)])-extra_ylim, max([np.max(P_avec_off),np.max(P_sans_off)])+extra_ylim)
plt.plot(t_domaine, P_sans_off[:,0]-1e6, linestyle="--", color="black", linewidth=2, label="Avec OFF") # Série OFF fantôme
plt.plot(t_domaine, P_sans_off[:,0]-1e6, linestyle="-", color="black", linewidth=2, label="Sans OFF") # Série OFF fantôme

plt.xlabel("Temps [s]")
plt.ylabel("Puissance de chauffage [W]")
plt.legend(loc="upper right")
plt.title("Puissance de chauffage au cours du temps")

plt.tight_layout()
plt.show()


In [ ]:
# Calculer l'énergie totale de chauffage pour chaque pièce.
E_sans_off = np.trapz(P_sans_off, t_domaine, axis=0)
E_avec_off = np.trapz(P_avec_off, t_domaine, axis=0)


# Polir un peu les résultats (mettre en kJ)
E_tot_sans_off = round(np.sum(E_sans_off)/1e3)
E_tot_avec_off = round(np.sum(E_avec_off)/1e3)

# Afficher un histogramme pour les énergies de chaque pièce
plt.figure(figsize=(10, 6))
plt.bar(range(1, L.shape[0] + 1), E_avec_off*1e-3, tick_label=[f"{i+1}" for i in range(L.shape[0])], color="darkBlue", alpha=0.4, label=f"Avec OFF : {E_tot_avec_off} kJ")
plt.bar(range(1, L.shape[0] + 1), E_sans_off*1e-3, tick_label=[f"{i+1}" for i in range(L.shape[0])], color="darkRed", alpha=0.4, label=f"Sans OFF : {E_tot_sans_off} kJ")
plt.xlabel("Pièce")
plt.ylabel("Énergie totale de chauffage [kJ]")
plt.title("Énergie totale de chauffage pour chaque pièce")
plt.legend()
plt.show()

#### Bref, je dois me rendre à l'évidence : avec ce modèle, mon coloc semble avoir raison!

# 4. Conclusion : Et si les simulations apprenaient d'elles-mêmes ?
---

Nous avons illustré au moyen de trois exemples en quoi les simulations numériques permettent de modéliser des systèmes complexes en physique. Mais que se passerait-il si, au lieu de coder explicitement chaque interaction, on laissait un modèle apprendre ces dynamiques?

C’est là qu’interviennent les réseaux neuronaux entraînés sur des modèles physiques. Plutôt que d’évaluer chaque équation pas à pas, on entraîne un réseau sur des données simulées. Une fois entraîné, le réseau tente de prédire l’évolution du système bien plus rapidement qu’une intégration numérique classique. 

En théorie, cette méthode est loin d'être farfelue, puisqu'on sait depuis 1989 [1] qu'un réseau neuronal peut agir à titre d'« approximateur universel » de toutes fonctions continues! Il est donc logique d'essayer de capturer la dynamique d'un système de cette manière.

Le potentiel de cette approche pour accélérer les simulations est déjà exploré dans plusieurs domaines, dont les sciences de la Terre [11]. 


**Les PINNs : le bac à sable du futur**

Les simulations basées sur les équations (comme celles du présent atelier) sont complémentaires à celles basées sur l’apprentissage machine. En combinant la rigueur des modèles physiques avec la puissance des réseaux neuronaux, on ouvre la voie à des simulations 
1) plus rapides,
2) moins énergivores,
3) détenant la capacité de généraliser sur de nouveaux cas.
   
On parle alors de « réseaux de neurones informés par la physique », ou PINNs (*Physics-informed neural networks*) en anglais. [8]

# 5. Références
---

[1] G. Cybenko, “Approximation by superpositions of a sigmoidal function,” Math. Control Signal Systems, vol. 2, no. 4, pp. 303–314, Dec. 1989, doi: 10.1007/BF02551274.

[2] “Centrifugal governor,” Wikipedia. Feb. 28, 2025. Accessed: Mar. 04, 2025. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Centrifugal_governor&oldid=1278155029

[3] M. D. Greenfield and B. Merker, “Coordinated rhythms in animal species, including humans: Entrainment from bushcricket chorusing to the philharmonic orchestra,” Neuroscience & Biobehavioral Reviews, vol. 153, p. 105382, Oct. 2023, doi: 10.1016/j.neubiorev.2023.105382.

[4] “Loi de refroidissement de Newton,” Wikipédia. Jul. 29, 2023. Accessed: Mar. 02, 2025. [Online]. Available: https://fr.wikipedia.org/w/index.php?title=Loi_de_refroidissement_de_Newton&oldid=206457504

[5] “Matrice laplacienne,” Wikipédia. Sep. 15, 2024. Accessed: Mar. 02, 2025. [Online]. Available: https://fr.wikipedia.org/w/index.php?title=Matrice_laplacienne&oldid=218663112

[6] E. W. Weisstein, “Positive Definite Matrix.” Accessed: Mar. 02, 2025. [Online]. Available: https://mathworld.wolfram.com/PositiveDefiniteMatrix.html

[7] “Régulateur à boules,” Wikipédia. Apr. 11, 2024. Accessed: Mar. 04, 2025. [Online]. Available: https://fr.wikipedia.org/w/index.php?title=R%C3%A9gulateur_%C3%A0_boules&oldid=214156361

[8] S. Cuomo, V. S. Di Cola, F. Giampaolo, G. Rozza, M. Raissi, and F. Piccialli, “Scientific Machine Learning Through Physics–Informed Neural Networks: Where we are and What’s Next,” J Sci Comput, vol. 92, no. 3, p. 88, Jul. 2022, doi: 10.1007/s10915-022-01939-z.

[9] R. Axelrod and W. D. Hamilton, “The evolution of cooperation,” Science, vol. 211, no. 4489, pp. 1390–1396, Mar. 1981, doi: 10.1126/science.7466396.

[10] “Thermal conduction,” Wikipedia. Feb. 25, 2025. Accessed: Mar. 02, 2025. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Thermal_conduction&oldid=1277574658

[11] C. Irrgang et al., “Towards neural Earth system modelling by integrating artificial intelligence in Earth system science,” Nat Mach Intell, vol. 3, no. 8, pp. 667–674, Aug. 2021, doi: 10.1038/s42256-021-00374-3.

[12] Veritasium, What Game Theory Reveals About Life, The Universe, and Everything, (2023). Accessed: Mar. 03, 2025. [Online Video]. Available: https://www.youtube.com/watch?v=mScpHTIi-kM




# ANNEXES
---

## Annexe A : Autres exemples de simulation
### i) Simulateur écologique
Un simulateur écologique basé sur l'individu a été développé dans le cadre du cours de *Dynamique non linéaire et chaos*. Il a été en mesure de mettre en évidence l'émergence de la collaboration entre des invidus qui interagissent entre eux dans un contexte de ressources limitées. 

<p float="left">
    <img src="inc/simul_chaos_lapino.png" width=600 height=600 />
    <img src="inc/simul_chaos_carte.png" width=600 height=600 />
</p>


Ce sujet a été largement exploré par des chercheurs en biologie et en théorie du jeu. Pour approfondir la question de l’évolution de la coopération, voici quelques ressources clés :
* *Le Gène égoïste*, par Richard Dawkins.
* *The Evolution of Cooperation*, par Robert Axelrod [9].
* [*Why is Cooperation So Hard*](https://www.youtube.com/watch?v=mScpHTIi-kM), par Veritasium [12].

### ii) Activité neuronale
De nombreuses simulations ont été réalisées en intégrant des modèles d'activité neuronale. Parmi celles-ci, certains modèles se sont avérés plus prometteurs que d’autres, en particulier une version modifiée de réseau neuronal récurrent à taux de décharge respectant la loi de Dale (c'est-à-dire avec des neurones spécialisés soit dans l'inhibition, soit dans l'excitation). Des performances remarquables ont néanmoins été obtenues avec une version matricielle du modèle de Kuramoto. Ces deux modèles ont pu reproduire la structure à l’échelle des régions du cerveau chez le poisson-zèbre (données gracieusement fournies par le laboratoire de Paul De Koninck), atteignant plus de 70 % de correspondance avec les données expérimentales (voir exemples de résultats ci-bas).

<p float="left">
<img src="inc/RNN_SCFC.png" width=600  />
<img src="inc/kuramoto_SCFC.png" width=600 />
</p>

Une application directe des simulations numériques est d'examiner dans quelle mesure les modèles théoriques d’activité neuronale peuvent simuler la structure neuronale à l’échelle des régions du cerveau. La connectivité fonctionnelle, mesurée par la corrélation des signaux neurophysiologiques entre régions cérébrales, est influencée par les connexions synaptiques sous-jacentes, les régions physiquement connectées ayant tendance à afficher des activités corrélées. Étant donné la difficulté d’obtenir des mesures directes de connectivité structurelle, il devient essentiel de développer des outils permettant de prédire la structure d’un réseau à partir de ses signaux d’activité fonctionnelle. Ainsi, ces modèles offrent un cadre pour explorer et affiner les liens entre activité fonctionnelle et structure neuronale.


## Annexe B : Dynamique de "relaxation" des températures en l'absence des radiateurs
Soit l'équation du système thermique sans radiateur
$$
C\frac{\text{d}\mathbf{T}(t)}{\text{d}t} = -L\mathbf{T}(t) \Longleftrightarrow \frac{\text{d}\mathbf{T}(t)}{\text{d}t} = -C^{-1}L\mathbf{T}(t).
$$
On suppose que la quantité matricielle $C^{-1}L$ est diagonalisable de manière à ce que $C^{-1}L = S\Lambda S{-1}$. Ainsi,
$$
\frac{\text{d}\mathbf{T}(t)}{\text{d}t} = -S\Lambda S^{-1}\mathbf{T}(t).
$$
En prenant $\tilde{\mathbf{T}}(t) = S{-1}\mathbf{T}$, l'équation se diagonalise, ce qui découple les $N$ équations sous-jacentes. On obtient alors
$$
S^{-1}\frac{\text{d}\mathbf{T}(t)}{\text{d}t} = -\Lambda S^{-1}\mathbf{T}(t) \implies \frac{\text{d}\tilde{\mathbf{T}}(t)(t)}{\text{d}t} = -\Lambda \tilde{\mathbf{T}}(t).
$$
Cette forme est directement intégrable. En posant $\mathbf{T}_0 \equiv \mathbf{T}(0)$ comme conditions initiales, sa solution va comme
$$
\tilde{\mathbf{T}}(t) = \tilde{\mathbf{T}}_0\exp\{-\Lambda t\} \implies \mathbf{T}(t) = \mathbf{T}_0\exp\{-\Lambda t\}.
$$
Ceci met en évidence que les températures sont vouées à la décroissance exponentielle. En effet, la matrice $C^{-1}$ étant diagonale et positive, son produit par $L$ ne sort pas les valeurs propres (réelles) de la positivité, signifiant alors que l'argument de l'exponentiel est strictement négatif (en raison du signe moins).

## Annexe C

### Recette : Comment réussir (ou pas) une simulation numérique

1. **Trouver un problème :** arroser les fleurs du tapis de la curiosité intellectuelle.
2. **Mathématiser le problème :** *soit un boeuf sphérique précédé d'une charrue...*
3. **Choisir des paramètres plausibles :** [*Itération 1*] partons à la cueillette de cerises, sans vergogne.
4. **Planifier une belle architecture** de code (digne du brutalisme, avec juste ce qu'il faut de chaos).
5. **Coder**, c’est-à-dire s'adonner à la chasse aux *bugs* en écoutant du [Georges Brassens](https://www.youtube.com/watch?v=oOPBnbNxLDM).
6. **Visualiser les résultats** (et choisir des couleurs menant à une hausse de crédibilité).

À la lumière des résultats, l'on peut juger de l'aptitude du modèle à bien décrire le phénomène. Des incréments en complexité peuvent alors être rajoutés si l'on juge que c'est nécessaire.